<a href="https://colab.research.google.com/github/Mohona77/Audio-Classification-SER-/blob/main/CAFE_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:

from huggingface_hub import notebook_login

notebook_login()

In [3]:

from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [4]:
%%capture
!pip install gdown

In [5]:

import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [6]:
%%capture
!gdown --id 1EZTvcHo2x3cim4uSTkfMHoagipfQtLak
time.sleep(5)

!unzip /content/FESC_CaFE.zip

In [8]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/CaFE/CaFE_splits/train.json",'r') as f:
    train_data = json.load(f)

with open("/content/CaFE/CaFE_splits/test.json",'r') as f:
    test_data = json.load(f)

In [9]:
train_data

{'09-J-1-4': {'wav': 'CaFE_48k/Happiness/Faible/09-J-1-4.wav',
  'length': 3.5731875,
  'emo': 'happiness'},
 '09-J-1-5': {'wav': 'CaFE_48k/Happiness/Faible/09-J-1-5.wav',
  'length': 3.8738125,
  'emo': 'happiness'},
 '04-J-1-6': {'wav': 'CaFE_48k/Happiness/Faible/04-J-1-6.wav',
  'length': 5.84675,
  'emo': 'happiness'},
 '02-J-1-1': {'wav': 'CaFE_48k/Happiness/Faible/02-J-1-1.wav',
  'length': 4.8348125,
  'emo': 'happiness'},
 '10-J-1-6': {'wav': 'CaFE_48k/Happiness/Faible/10-J-1-6.wav',
  'length': 4.38675,
  'emo': 'happiness'},
 '10-J-1-4': {'wav': 'CaFE_48k/Happiness/Faible/10-J-1-4.wav',
  'length': 4.6013125,
  'emo': 'happiness'},
 '02-J-1-3': {'wav': 'CaFE_48k/Happiness/Faible/02-J-1-3.wav',
  'length': 4.6151875,
  'emo': 'happiness'},
 '04-J-1-4': {'wav': 'CaFE_48k/Happiness/Faible/04-J-1-4.wav',
  'length': 4.626,
  'emo': 'happiness'},
 '09-J-1-6': {'wav': 'CaFE_48k/Happiness/Faible/09-J-1-6.wav',
  'length': 3.7488125,
  'emo': 'happiness'},
 '04-J-1-5': {'wav': 'CaFE_

In [10]:
test_data

{'11-J-1-6': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-6.wav',
  'length': 5.1439375,
  'emo': 'happiness'},
 '11-J-1-4': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-4.wav',
  'length': 4.9079375,
  'emo': 'happiness'},
 '11-J-1-5': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-5.wav',
  'length': 4.6163125,
  'emo': 'happiness'},
 '11-J-1-1': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-1.wav',
  'length': 4.2954375,
  'emo': 'happiness'},
 '11-J-1-2': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-2.wav',
  'length': 4.0303125,
  'emo': 'happiness'},
 '11-J-1-3': {'wav': 'CaFE_48k/Happiness/Faible/11-J-1-3.wav',
  'length': 4.2556875,
  'emo': 'happiness'},
 '12-J-1-6': {'wav': 'CaFE_48k/Happiness/Faible/12-J-1-6.wav',
  'length': 5.267,
  'emo': 'happiness'},
 '12-J-1-5': {'wav': 'CaFE_48k/Happiness/Faible/12-J-1-5.wav',
  'length': 5.34375,
  'emo': 'happiness'},
 '12-J-1-4': {'wav': 'CaFE_48k/Happiness/Faible/12-J-1-4.wav',
  'length': 5.4876875,
  'emo': 'happiness'},
 '12-J-1-1': {'wav': 'CaF

In [11]:
train_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in train_data.items()
]
train_dataset = Dataset.from_list(train_samples_list)


In [12]:
test_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in test_data.items()
]
test_dataset = Dataset.from_list(test_samples_list)

In [13]:
dataset = DatasetDict({
    'train' : train_dataset,
    'test' : test_dataset,
    'dev' : test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 780
    })
    test: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 156
    })
    dev: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 156
    })
})

In [14]:
def add_base_path(example):
    example['wav'] = '/content/CaFE/' + example['wav']  # Adjust column name if it's not 'wav'
    return example

dataset = dataset.map(add_base_path, batched=False)
dataset['train'][0]

Map:   0%|          | 0/780 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

{'id': '09-J-1-4',
 'wav': '/content/CaFE/CaFE_48k/Happiness/Faible/09-J-1-4.wav',
 'length': 3.5731875,
 'emo': 'happiness'}

In [15]:
import librosa
def load_audio(example):
  audio_array, sample_rate = librosa.load(example['wav'], sr=16000)
  example["audio_array"] = audio_array
  example["sample_rate"] = sample_rate
  return example

In [16]:
dataset = dataset.map(load_audio, batched=False)
dataset

Map:   0%|          | 0/780 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 780
    })
    test: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 156
    })
    dev: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 156
    })
})

In [17]:
dataset['train'][0].keys()

dict_keys(['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'])

In [18]:
import torch
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor, TrainingArguments, Trainer
!pip install datasets --upgrade


In [32]:
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import recall_score, confusion_matrix, classification_report
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import Wav2Vec2Processor
from torch.optim import Adam

# Define emotion labels and mappings
labels = ['anger', 'happiness', 'neutral', 'sadness']
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for i, label in enumerate(labels)}

# Initialize processor for Wav2Vec2
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")

# Function to load data from JSON files


def load_data_from_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    audio_files = []
    labels = []

    # Access the "data" key, which contains a list of files
    for file_data in data["data"]:
        # Each file has "wav" and "emo" under the first key (file id)
        file_id = list(file_data.keys())[0]
        audio_files.append(file_data[file_id]["wav"])
        labels.append(label2id[file_data[file_id]["emo"]])

    return audio_files, labels

# Load train and test data
train_audio_files, train_labels = load_data_from_json("/content/CaFE/CaFE_splits/train.json")
test_audio_files, test_labels = load_data_from_json("/content/CaFE/CaFE_splits/test.json")

# Custom dataset class
class EmotionDataset(Dataset):
    def __init__(self, audio_files, labels):
        self.audio_files = audio_files
        self.labels = labels

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        # Load audio data
        audio_data, sample_rate = torchaudio.load(self.audio_files[idx])
        # Preprocess with Wav2Vec2 processor
        input_values = processor(audio_data.squeeze(), sampling_rate=sample_rate).input_values[0]
        label = self.labels[idx]
        return {"input_values": input_values, "label": label}

# Prepare datasets
train_dataset = EmotionDataset(train_audio_files, train_labels)
test_dataset = EmotionDataset(test_audio_files, test_labels)

# Custom collate function for padding
def collate_fn(batch):
    batch = sorted(batch, key=lambda x: len(x["input_values"]), reverse=True)
    inputs = [torch.tensor(example["input_values"]) for example in batch]
    labels = [int(example["label"]) for example in batch]
    inputs = pad_sequence(inputs, batch_first=True)
    return {"input_values": inputs, "label": torch.tensor(labels)}

# Train function
def train(model, train_loader, optimizer, loss_fn, device):
    model.train()
    total_predictions = 0
    correct_predictions = 0
    total_train_loss = 0
    all_actual_labels = []
    all_predictions = []

    for batch in train_loader:
        inputs, labels = batch['input_values'].to(device), batch['label'].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs.logits
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
        total_train_loss += loss.item()
        all_actual_labels.extend(labels.tolist())
        all_predictions.extend(predicted.tolist())

    train_accuracy = correct_predictions / total_predictions
    unweighted_recall = recall_score(all_actual_labels, all_predictions, average='macro')
    weighted_recall = recall_score(all_actual_labels, all_predictions, average='weighted')
    return {
        "train_accuracy": train_accuracy,
        "train_loss": total_train_loss / len(train_loader),
        "unweighted_recall": unweighted_recall,
        "weighted_recall": weighted_recall
    }

# Test function
def test(model, test_loader, loss_fn, device):
    model.eval()
    total_predictions = 0
    correct_predictions = 0
    total_test_loss = 0
    all_actual_labels = []
    all_predictions = []

    with torch.no_grad():
        for batch in test_loader:
            inputs, labels = batch['input_values'].to(device), batch['label'].to(device)
            outputs = model(inputs)
            outputs = outputs.logits
            loss = loss_fn(outputs, labels)
            total_test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
            all_actual_labels.extend(labels.tolist())
            all_predictions.extend(predicted.tolist())

    test_accuracy = correct_predictions / total_predictions
    unweighted_recall = recall_score(all_actual_labels, all_predictions, average='macro')
    weighted_recall = recall_score(all_actual_labels, all_predictions, average='weighted')
    conf_matrix = confusion_matrix(all_actual_labels, all_predictions)
    print("Confusion Matrix:")
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
    plt.show()
    print("Classification Report:")
    print(classification_report(all_actual_labels, all_predictions))
    return {
        "test_accuracy": test_accuracy,
        "test_loss": total_test_loss / len(test_loader),
        "unweighted_recall": unweighted_recall,
        "weighted_recall": weighted_recall
    }

# Training and evaluation function
def train_and_evaluate(model, train_loader, test_loader, optimizer, loss_fn, device, epochs=10):
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        train_metrics = train(model, train_loader, optimizer, loss_fn, device)
        print(f"Train Loss: {train_metrics['train_loss']:.4f}, Train Accuracy: {train_metrics['train_accuracy']:.4f}")
        test_metrics = test(model, test_loader, loss_fn, device)
        print(f"Test Loss: {test_metrics['test_loss']:.4f}, Test Accuracy: {test_metrics['test_accuracy']:.4f}")

# Setup for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Wav2Vec2ForSequenceClassification.from_pretrained("facebook/wav2vec2-base", num_labels=len(labels))
model.to(device)
optimizer = Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

train_loader = DataLoader(train_dataset, batch_size=8, collate_fn=collate_fn, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, collate_fn=collate_fn, shuffle=False)

# Train and evaluate the model
train_and_evaluate(model, train_loader, test_loader, optimizer, loss_fn, device, epochs=10)


tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

KeyError: 'data'